<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Figure 3: Dependence of model performance on forest type

_Authors: Oriana Chegwidden and Jeremy Freeman_

The methods below conduct the analyses to recreate Figure 3 included the
manuscript <<< insert doi >>>. In this analysis we mean forest group type.


In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import scipy as sp
from carbonplan_forests import load, setup, plot, fit, utils, prepare, collect
import xarray as xr
from carbonplan_forests.utils import get_store
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from carbonplan_forests import load, setup, plot, fit, utils, prepare, collect
from showit import image
import scipy as sp
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import regionmask as rm
from scipy.stats import binom
import altair as alt

from palettable.colorbrewer.sequential import YlOrRd_9

In [ ]:
coarsen = 4
store = "az"
tlim = ("1984", "2018")

In [ ]:
mask = load.mask(store=store, year=2001)

In [ ]:
nlcd = load.nlcd(store=store, year=2001, coarsen=coarsen, mask=mask)
nftd = load.nftd(store=store, area_threshold=1500, coarsen=coarsen, mask=mask)

In [ ]:
mtbs = load.mtbs(store=store, coarsen=coarsen, tlim=tlim, mask=mask)
# give it the same x/y coords as nftd (and nlcd and climate)
# this re-assignment will help us with some masking lower down
mtbs = mtbs.assign_coords({"x": nftd.x, "y": nftd.y})

In [ ]:
historical_ds = xr.open_zarr(
    get_store("carbonplan-scratch", "data/fire.zarr")
).assign_coords({"y": mtbs.y, "x": mtbs.x})

In [ ]:
alt.data_transformers.disable_max_rows()

In [ ]:
all_evals = []
all_stats = []
for band in nftd.band.values:
    stats, chart = plot.fire.full_eval(
        mtbs * nftd.sel(band=band),
        historical_ds * nftd.sel(band=band),
        data_var="monthly",
        model_var="historical",
        clim=(-0.001, 0.001),
        cmap="purplegreen",
        percentage=False,
        clabel="Diff (Mod-Obs) [-]",
    )
    all_stats.append(stats)
    all_evals.append(chart)
alt.vconcat(*all_evals)

In [ ]:
test = mtbs.groupby("time.year").sum().mean(dim="year") * nftd

In [ ]:
mtbs_band_probs = test.mean(["x", "y"]).to_dataframe()

In [ ]:
mtbs_band_probs.loc[100]["monthly"]

In [ ]:
df_stats = pd.DataFrame(
    columns=[
        "forest group type",
        "annual correlation",
        "seasonal correlation",
        "spatial correlation",
        "group-averaged risk",
    ]
)
for row, (band, performances) in enumerate(zip(nftd.band.values, all_stats)):
    df_stats.loc[row] = [
        str(band),
        performances["annual"],
        performances["seasonal"],
        performances["spatial"],
        mtbs_band_probs.loc[band]["monthly"],
    ]

In [ ]:
alt.Chart(df_stats).mark_bar().encode(
    x="forest group type", y="annual correlation", color="group-averaged risk"
) | alt.Chart(df_stats).mark_bar().encode(
    x="forest group type", y="seasonal correlation", color="group-averaged risk"
) | alt.Chart(
    df_stats
).mark_bar().encode(
    x="forest group type", y="spatial correlation", color="group-averaged risk"
)